In [9]:

import warnings
warnings.filterwarnings('ignore')

In [1]:
# Import necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Model Building

In [2]:
#importing the necessary packages.
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.metrics import f1_score, classification_report
#from scikitplot.metrics import plot_confusion_matrix
#from scikitplot.classifiers import plot_feature_importances

#importing model packages.
import xgboost as xgb
import lightgbm as lgb
import mlflow

In [3]:
# Set the MLflow experiment name
mlflow.set_experiment("XGBoost_GridSearchCV_Experiment")

<Experiment: artifact_location='file:///Users/madhuri/Documents/instacart_analysis/mlruns/409450851093224200', creation_time=1741387367950, experiment_id='409450851093224200', last_update_time=1741387367950, lifecycle_stage='active', name='XGBoost_GridSearchCV_Experiment', tags={}>

In [4]:
data_train = pd.read_csv('./data/train_data.csv')

In [5]:
#Creating X and y variables.
X = data_train.drop(['reordered', 'uxp_ratio_last_five'], axis=1)
y = data_train.reordered

#splitting dataset into train and test split.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

In [6]:
# Define the XGBoost model
model = xgb.XGBClassifier(random_state=42)

In [7]:
# Define the hyperparameter grid
param_grid = {
    'max_depth': [3, 5],
    'learning_rate': [0.01, 0.1],
    'n_estimators': [100, 150],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
}

In [8]:
# Start an MLflow run
with mlflow.start_run():
    # Initialize GridSearchCV
    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        scoring='f1',
        cv=5,
        verbose=1,
        n_jobs=-1
    )

    # Fit the model
    grid_search.fit(X_train, y_train)

    # Log the best parameters and metrics
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_

    mlflow.log_params(best_params)
    mlflow.log_metric("best_cv_f1_score", best_score)

    # Evaluate the best model on the test set
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)
    test_f1_score = f1_score(y_test, y_pred)

    mlflow.log_metric("test_f1_score", test_f1_score)

    # Log the best model
    mlflow.xgboost.log_model(best_model, "best_model")

    print(f"Best CV F1 Score: {best_score}")
    print(f"Test F1 Score: {test_f1_score}")
    print(f"Best Parameters: {best_params}")

Fitting 5 folds for each of 32 candidates, totalling 160 fits


/opt/anaconda3/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [12:49:31] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
2025/03/08 12:49:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Best CV F1 Score: 0.2679153847536591
Test F1 Score: 0.26590361025816533
Best Parameters: {'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'subsample': 1.0}


Why am i getting such a low F-1 score? - like 0.26 - something is wrong, I need to check it out.